<a href="https://colab.research.google.com/github/vifirsanova/hse-python-course/blob/main/compression/quantization_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

In [26]:
# загружаем данные
(X_train, y_train), (X_test, y_test) = datasets.mnist.load_data()

# регуляризация изображений (значения в диапазоне от 0 до 1)
X_train, X_test = X_train / 255.0, X_test / 255.0

# подгоняем размер изображений под модель
X_train = X_train.reshape(-1, 28 * 28)
X_test = X_test.reshape(-1, 28 * 28)

In [27]:
model = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=(784,)),
    layers.Dropout(0.2),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8392 - loss: 0.5577 - val_accuracy: 0.9498 - val_loss: 0.1691
Epoch 2/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9477 - loss: 0.1799 - val_accuracy: 0.9638 - val_loss: 0.1227
Epoch 3/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9629 - loss: 0.1237 - val_accuracy: 0.9714 - val_loss: 0.0945
Epoch 4/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9702 - loss: 0.0997 - val_accuracy: 0.9741 - val_loss: 0.0847
Epoch 5/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9741 - loss: 0.0827 - val_accuracy: 0.9758 - val_loss: 0.0779
Epoch 6/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.9775 - loss: 0.0745 - val_accuracy: 0.9768 - val_loss: 0.0770
Epoch 7/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9809 - loss: 0.0616 - val_accuracy: 0.9762 - val_loss: 0.0736
Epoch 8/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9816 - loss: 0.0580 - val_accuracy: 0.

In [28]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc * 100:.2f}%')

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9759 - loss: 0.0807
Test accuracy: 97.98%


Конвертируем к виду моделиTensorFlow Lite

In [29]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# сохраняем исходную модель
with open('mnist_model.tflite', 'wb') as f:
    f.write(tflite_model)

# применяем квантизацию
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()

with open('mnist_model_quant.tflite', 'wb') as f:
    f.write(tflite_quant_model)

INFO:tensorflow:Assets written to: /tmp/tmpocloqyqr/assets


Saved artifact at '/tmp/tmpocloqyqr'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 784), dtype=tf.float32, name='keras_tensor_4')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  137043483990976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137043483988512: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137043992887536: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137043484720848: TensorSpec(shape=(), dtype=tf.resource, name=None)


INFO:tensorflow:Assets written to: /tmp/tmp36qfh7tg/assets


Saved artifact at '/tmp/tmp36qfh7tg'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 784), dtype=tf.float32, name='keras_tensor_4')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  137043483990976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137043483988512: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137043992887536: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137043484720848: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [30]:
# загрузка квантованной модели
interpreter = tf.lite.Interpreter(model_path='mnist_model_quant.tflite')
interpreter.allocate_tensors()

# входные и выходные матрицы
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# оценка модели
correct = 0
total = 0

for i in range(len(X_test)):
    input_data = np.expand_dims(X_test[i], axis=0).astype(np.float32)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    predicted_label = np.argmax(output_data)
    true_label = y_test[i]
    if predicted_label == true_label:
        correct += 1
    total += 1

quantized_test_acc = correct / total
print(f'Quantized Test accuracy: {quantized_test_acc * 100:.2f}%')

Quantized Test accuracy: 97.95%


Сравним размеры моделей

In [31]:
import os

unquantized_model_size = os.path.getsize('mnist_model.tflite')
print(f'Unquantized model size: {unquantized_model_size} bytes')

Unquantized model size: 408792 bytes


In [32]:
quantized_model_size = os.path.getsize('mnist_model_quant.tflite')
print(f'Unquantized model size: {unquantized_model_size} bytes')

Unquantized model size: 408792 bytes


Сравним скорость инференса

In [33]:
import time

interpreter_unquantized = tf.lite.Interpreter(model_path='mnist_model.tflite')
interpreter_unquantized.allocate_tensors()

input_details_unquantized = interpreter_unquantized.get_input_details()
output_details_unquantized = interpreter_unquantized.get_output_details()

# измеряем время инференса
start_time = time.time()
for i in range(len(X_test)):
    input_data = np.expand_dims(X_test[i], axis=0).astype(np.float32)
    interpreter_unquantized.set_tensor(input_details_unquantized[0]['index'], input_data)
    interpreter_unquantized.invoke()
end_time = time.time()

unquantized_inference_time = end_time - start_time
print(f'Unquantized model inference time: {unquantized_inference_time:.4f} seconds')

Unquantized model inference time: 0.9270 seconds


In [34]:
interpreter_quantized = tf.lite.Interpreter(model_path='mnist_model_quant.tflite')
interpreter_quantized.allocate_tensors()

input_details_quantized = interpreter_quantized.get_input_details()
output_details_quantized = interpreter_quantized.get_output_details()

start_time = time.time()
for i in range(len(X_test)):
    input_data = np.expand_dims(X_test[i], axis=0).astype(np.float32)
    interpreter_quantized.set_tensor(input_details_quantized[0]['index'], input_data)
    interpreter_quantized.invoke()
end_time = time.time()

quantized_inference_time = end_time - start_time
print(f'Quantized model inference time: {quantized_inference_time:.4f} seconds')

Quantized model inference time: 0.2570 seconds
